# Analysis

This notebook provides several analyses to gain deeper intuition into system performance.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path
import IPython.display as ipd
import import_ipynb
import system_utils

## Visualizing errors over time

The first analysis is to simply visualize the magnitude of alignment errors over time for a specific concerto movement.  The code below allows you to specify a list of `(system, fullmix_id, tsm_factor)` settings to compare on the same plot.

In [ ]:
def calculate_errors_over_time(eval_dir, scenarios_summary, fullmix_id, tsm_factor):
    '''
    Calculate the alignment error vs measure number for a given full mix recording & TSM factor.
    Measures that are not evaluated will be displayed as having zero error.
    
    Inputs
    eval_dir: the eval directory containing alignment error information
    scenarios_summary: filepath specifying the scenarios.summary file
    fullmix_id: id specifying the full mix recording of interest, e.g. 'rach2_mov1_PO2'
    tsm_factor: a string specifying the TSM factor, e.g. 'tsm0.80'
    
    Outputs
    alignErrors: alignment error at the downbeat of each measure
    measures: array containing indices of corresponding measures
    '''
    
    # parse relevant files
    with open(f'{eval_dir}/errs.pkl', 'rb') as f:
        errors_info = pickle.load(f) # key: scenario_id, value: (errs, measNums)
    scenarios_info = system_utils.get_scenario_info(scenarios_summary) # key: scenario_id, value: dict with scenario info
    
    # aggregate data
    data = {} # key: measure number, value: alignment error
    for scenario_id in scenarios_info:
        if fullmix_id in scenarios_info[scenario_id]['po'] and tsm_factor in scenarios_info[scenario_id]['p']:
            errs, measNums = errors_info[scenario_id] # only contains evaluation measures
            for err, measNum in zip(errs, measNums):
                data[measNum] = err
    
    # assign 0 error to non-evaluated measures
    maxMeasNum = int(np.max(list(data.keys())))
    alignErrors = []
    for i in range(1,maxMeasNum+1):
        if i in data:
            alignErrors.append(data[i])
        else:
            alignErrors.append(0)
    
    return np.array(alignErrors), np.arange(1,maxMeasNum+1)

In [ ]:
def plot_errors_over_time(eval_dirs, fullmix_ids, tsm_factors, scenarios_summary, subtitles = None):
    '''
    Plots the alignment error vs measure number for a set of (system, full mix, TSM factor) tuples.
    Measures that are not evaluated will be displayed as having zero error.
    
    Inputs
    eval_dirs: list of eval directories, specifies the system for each subplot
    fullmix_ids: list of strings specifying the full mix id (e.g. 'rach2_mov1_PO2') for each subplot
    tsm_factors: list of strings specifying the TSM factor (e.g. 'tsm0.80') for each subplot
    scenarios_summary: filepath specifying the scenarios.summary file
    subtitles: if specified, specifies the subtitles on each subplot
    
    Generates subplots comparing the alignment error vs measure number for each specified setting.
    '''
    fig, axs = plt.subplots(len(eval_dirs), 1, figsize = (8, 8), sharex=True, sharey=True)    
    
    for i, (eval_dir, fullmix_id, tsm_factor) in enumerate(zip(eval_dirs, fullmix_ids, tsm_factors)):
        
        errs, measNums = calculate_errors_over_time(eval_dir, scenarios_summary, fullmix_id, tsm_factor)    
        axs[i].plot(measNums, errs)
        if subtitles is not None:
            axs[i].set_title(subtitles[i])
        axs[i].grid(linestyle='--')
        axs[i].set_ylim([-3, 3])
        
    fig.supxlabel('Measure Number')
    fig.supylabel('Alignment Error (sec)')

In [ ]:
eval_dirs = ['eval/match', 'eval/match','eval/offlineDTW','eval/offlineDTW'] # eval directory to visualize
fullmix_ids = ['rach2_mov1_PO1','rach2_mov1_PO2','rach2_mov1_PO1','rach2_mov1_PO2']
#tsm_factors = ['tsm0.80', 'tsm0.90','tsm1.00','tsm1.11','tsm1.25']
tsm_factors = ['tsm1.00'] *4
SCENARIOS_SUMMARY = 'scenarios/scenarios.summary'
subtitles = ['MATCH - M1', 'MATCH - M2', 'FSVE DTW - M1', 'FSVE DTW - M2']
plot_errors_over_time(eval_dirs, fullmix_ids, tsm_factors, SCENARIOS_SUMMARY, subtitles)

## Generating sonifications

The second analysis is to sonify estimated alignments.  We generate an audio file that contains an unmodified recording on the left channel and a time-scale modified version of the other recording on the right channel, where time-scale modification is applied in order to synchronize the two recordings.

In [ ]:
import sonify_tools

### Sonifying specific alignments

The cells below generate sonifications of the P-O, P-PO, and O-PO alignments for a given scenario.

In [ ]:
def getCacheDir(scenario_id, system_name):
    '''
    Given a scenario id and system id, determines the filepath to the cache directory.
    
    Inputs
    scenario_id: id of the scenario to process
    system_name: id of the system to process
    '''
    info_file = f'scenarios/{scenario_id}/scenario.info'
    d = system_utils.get_scenario_info(info_file)
    o_basename = os.path.splitext(os.path.basename(d['o']))[0] # e.g. rach2_mov1_O1
    po_basename = os.path.splitext(os.path.basename(d['po']))[0] # e.g. rach2_mov1_PO1
    cache_dir = f'experiments/{system_name}/cache/{o_basename}_' + po_basename.split('_')[-1]
    return cache_dir

In [ ]:
### edit below ###
scenario_id = 's171'
system_name = 'offlineDTW'
downsample = 20
sr = 22050
hop_samples = 512
##################

In [ ]:
### sonifying P-O alignment
outfile = f'{scenario_id}_p_o_align.wav'
hop_len = None # set to None if warping path is already expressed in seconds
audiofile1 = f'scenarios/{scenario_id}/p.wav'
audiofile2 = f'scenarios/{scenario_id}/o.wav'
align_file = f'experiments/{system_name}/{scenario_id}/hyp.npy' # p-o alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

In [ ]:
### sonifying P-PO alignment
outfile = f'{scenario_id}_p_po_align.wav'
hop_len = hop_samples / sr
audiofile1 = f'scenarios/{scenario_id}/p.wav'
audiofile2 = f'scenarios/{scenario_id}/po.wav'
align_file = f'experiments/{system_name}/{scenario_id}/p_po_align.npy' # p-po alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

In [ ]:
### sonifying O-PO alignment
outfile = f'{scenario_id}_o_po_align.wav'
hop_len = hop_samples / sr
audiofile1 = f'scenarios/{scenario_id}/o.wav'
audiofile2 = f'scenarios/{scenario_id}/po.wav'
align_file = getCacheDir(scenario_id, system_name) + '/o_po_align.npy' # o-po alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

Listen to a recording:

In [ ]:
ipd.Audio(f'{scenario_id}_p_po_align.wav')

### Batch Sonification

The following two cells generate P-O sonifications for all scenarios:

In [ ]:
SCENARIOS_DIR = 'scenarios'
EXP_DIR = 'experiments/offlineDTW'
SONIFY_DIR = f'{EXP_DIR}/sonify'
downsample = 20
hop_len = 512./22050

In [ ]:
sonify_tools.sonifyWithTSMSync_batch(SCENARIOS_DIR, EXP_DIR, downsample, hop_len, SONIFY_DIR)